In [1]:
mutable struct Node{T} 
    data::Union{T, Nothing}
    prev::Union{Node{T}, Nothing}
    next::Union{Node{T}, Nothing}
end

mutable struct Buffer
    nodes::Vector{Node}
    length::Int
    current::Node
end

In [2]:
import Base: insert!, delete!, length

function circlist(data::T; capacity = 100) where T
    nodes = [Node{T}(nothing, nothing, nothing) for _ in 1:capacity]
    n = nodes[1]
    n.data = data
    n.prev = n
    n.next = n
    return Buffer(nodes, 1, n)
end

function allocate!(buf::Buffer)
    buf.length += 1
    return buf.nodes[buf.length]
end

# insert a new node after the current node, returns new node
function insert!(buf::Buffer, data) 
    cl = buf.current
    
    n = allocate!(buf)  # make a new node and arrange prev/next pointers
    n.data = data
    n.prev = cl
    n.next = cl.next

    cl.next = n         # fix prev node's next pointer
    n.next.prev = n     # fix next node's prev pointer
    
    buf.current = n     # move pointer to newly inserted node
    return buf
end

# delete current node, returns previous node
# warning: removed nodes are not reclaimed from memory for simplicity reasons
function delete!(buf::Buffer)
    cl = buf.current
    cl.prev.next = cl.next   # fix prev node's next pointer
    cl.next.prev = cl.prev   # fix next node's prev pointer
    buf.current = cl.prev    # reset buffer's current pointer to prev
    return buf
end

# navigate forward/backward in the circular list
function shift!(buf::Buffer, steps::Int, direction::Symbol)
    for i in 1:steps
        if direction == :forward 
            buf.current = buf.current.next
        elseif direction == :backward
            buf.current = buf.current.prev
        end
    end
    return buf
end

forward!(buf::Buffer) = shift!(buf, 1, :forward)
backward!(buf::Buffer) = shift!(buf, 1, :backward)
;

In [3]:
using Dates, Test

function play(num_players, last_marble; chunk = 100_000)
    leaderboard = fill(0, num_players)
    game = curr_marble = circlist(0, capacity = last_marble)
    marble = 0
    while marble < last_marble
        marble += 1
        marble % chunk == 0 && (println(Dates.now(), ": marble $marble processed"), flush(stdout))
        if marble % 23 == 0
            player = (marble % num_players) + 1
            leaderboard[player] += marble   # win current marble
            curr_marble = shift!(curr_marble, 7, :backward)
            leaderboard[player] += curr_marble.current.data  # win the other marble
            curr_marble = delete!(curr_marble)
            curr_marble = forward!(curr_marble)
        else
            curr_marble = forward!(curr_marble)
            curr_marble = insert!(curr_marble, marble)
        end
    end
    return (scores = leaderboard, 
        highest = maximum(leaderboard),
        num_players = num_players,
        last_marble = last_marble
        )
end

@test play(9, 25).highest == 32
@test play(10, 1618).highest == 8317
@time play(431, 70950)

  0.254985 seconds (377.76 k allocations: 11.934 MiB, 2.89% gc time)


(scores = [375685, 302467, 327051, 314203, 345826, 347920, 378222, 306633, 289693, 318892  …  305662, 335382, 305703, 359932, 371519, 389490, 307795, 293451, 318929, 338933], highest = 404611, num_players = 431, last_marble = 70950)

In [4]:
@time play(431, 7095000)

2018-12-09T02:28:23.277: marble 100000 processed
2018-12-09T02:28:23.663: marble 200000 processed
2018-12-09T02:28:23.943: marble 300000 processed
2018-12-09T02:28:24.223: marble 400000 processed
2018-12-09T02:28:24.506: marble 500000 processed
2018-12-09T02:28:25.587: marble 600000 processed
2018-12-09T02:28:25.869: marble 700000 processed
2018-12-09T02:28:26.139: marble 800000 processed
2018-12-09T02:28:26.409: marble 900000 processed
2018-12-09T02:28:26.678: marble 1000000 processed
2018-12-09T02:28:26.949: marble 1100000 processed
2018-12-09T02:28:27.285: marble 1200000 processed
2018-12-09T02:28:27.662: marble 1300000 processed
2018-12-09T02:28:27.976: marble 1400000 processed
2018-12-09T02:28:28.258: marble 1500000 processed
2018-12-09T02:28:28.562: marble 1600000 processed
2018-12-09T02:28:28.863: marble 1700000 processed
2018-12-09T02:28:29.15: marble 1800000 processed
2018-12-09T02:28:29.461: marble 1900000 processed
2018-12-09T02:28:29.745: marble 2000000 processed
2018-12-09

(scores = [3329318022, 3338084362, 3341579397, 3336818909, 3347592587, 3340225602, 3338953147, 3343797117, 3340419992, 3340629970  …  3325297181, 3324569457, 3319641009, 3310406271, 3312373465, 3320100167, 3324828503, 3324005306, 3331266930, 3321826335], highest = 3350093681, num_players = 431, last_marble = 7095000)